In [1]:
import os, shutil, random
from turtle import mode
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

Oversampling

In [5]:
# Define the path to your "Moderate demented" images
source_dir = 'Binary_TTV_over_under_all\\test\Demented'


# Create a directory to save the augmented images
augmented_dir = 'Binary_TTV_over_under_all\\test\\Demented\Augmented'
os.makedirs(augmented_dir, exist_ok=True)

# Initialize the ImageDataGenerator with augmentation settings
datagen = ImageDataGenerator(
    rescale=128./255,              # Rescale pixel values to [0, 1]
    brightness_range=[0.5, 1.5],   # Random brightness adjustment
    zoom_range=[0.8, 1.2],         # Random zooming
    fill_mode='nearest',           # Fill mode for filling pixels after augmentation
    horizontal_flip=True,          # Random horizontal flipping
    vertical_flip=True,            # Random vertical flipping
    rotation_range=30,             # Random rotations
    shear_range=0.3,               # Random shearing
)

# List all images in the "Moderate demented" directory
image_files = [i for i in os.listdir(source_dir) if i.endswith('.jpg')]
print(len(image_files))
# Set the target number of augmented images (e.g., 700)

target_count = 3600

2882


In [6]:
# Loop until the target count is reached
generated_count = 0
print(f'Generating {target_count} augmented images...', end='\n\n')
with tqdm(total=target_count, desc="Augmentation Progress") as pbar:
    while generated_count < target_count:
        # Randomly select an image from the original dataset
        image_file = np.random.choice(image_files)
        image_path = os.path.join(source_dir, image_file)

        # Load the image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure it's in RGB format

        # Generate augmented images using the data generator
        aug_iter = datagen.flow(np.expand_dims(image, axis=0), batch_size=1)
        augmented_images = [next(aug_iter)[0].astype(np.uint8) for _ in range(5)]  # Create 5 augmented images per original

        # Save the augmented images
        for i, aug_image in enumerate(augmented_images):
            aug_filename = f'augmented_{generated_count + i}.jpg'
            aug_image_path = os.path.join(augmented_dir, aug_filename)
            cv2.imwrite(aug_image_path, cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR))

        # Update the count of generated images
        generated_count += len(augmented_images)
        # if generated_count % 6000 == 0:
        #     print(f'{generated_count} images generated...')
        pbar.update(len(augmented_images))

print(f'Generated {generated_count} augmented images.')

Generating 3600 augmented images...



Augmentation Progress: 100%|██████████| 3600/3600 [01:10<00:00, 50.85it/s]

Generated 3600 augmented images.


Undersampling

In [7]:
# Path to the folders
non_demented_folder = "Binary_TTV_over_under_all/test/NonDemented_original/"
output_folder = "Binary_TTV_over_under_all/test/NonDemented/"  # Specify the folder where the downsampled images will be stored

# Target number of images
target_count = 6482

# Get the list of non-demented files
non_demented_files = os.listdir(non_demented_folder)
print(len(non_demented_files))

# Randomly select files to achieve the target count
selected_non_demented_files = random.sample(non_demented_files, target_count)
print(len(selected_non_demented_files))

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Copy the selected non-demented files to the output folder
with tqdm(total=target_count, desc="Undersampling Progress") as pbar:
    for file_name in selected_non_demented_files:
        source_path = os.path.join(non_demented_folder, file_name)
        destination_path = os.path.join(output_folder, file_name)
        # if pbar.n % 5000 == 0:
        #     print(f"Copying {source_path} to {destination_path}")
        shutil.copy(source_path, destination_path)
        pbar.update(1)

print(f"Downsampling completed. {target_count} non-demented images saved to {output_folder}.")


10083
6482


Undersampling Progress: 100%|██████████| 6482/6482 [00:31<00:00, 207.64it/s]

Downsampling completed. 6482 non-demented images saved to Binary_TTV_over_under_all/test/NonDemented/.
